<a href="https://colab.research.google.com/github/cric96/DL-exercise/blob/main/gnn_by_hand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install numpy
!pip install tensorflow
!pip install spektral

import numpy as np
import tensorflow as tf
import spektral

     |████████████████████████████████| 133kB 5.2MB/s 


In [2]:
from spektral.datasets.citation import Cora
dataset = Cora()
graph = dataset[0]
adj, features, labels = graph.a, graph.x, graph.y
train_mask, val_mask, test_mask = dataset.mask_tr, dataset.mask_va, dataset.mask_te
 
features = features.astype("float32")
adj = adj.astype("float32")

print(features.shape)
print(adj.shape)
print(labels.shape)

print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))

(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [52]:
def masked_softmax_cross_entropy(logits, labels, mask):
  loss = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels)
  mask = tf.cast(mask, dtype = tf.float32)
  mask /= tf.reduce_mean(mask)
  loss *= mask
  return tf.reduce_mean(loss)


def masked_accuracy(logits, labels, mask):
  correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
  accuracy_all = tf.cast(correct_prediction, tf.float32)
  mask = tf.cast(mask, dtype = tf.float32)
  mask /= tf.reduce_mean(mask)
  accuracy_all *= mask
  return tf.reduce_mean(accuracy_all)

In [74]:
def gnn(fts, adj, transform, activation):
  seq_fts = transform(fts)
  ret_fts = tf.matmul(adj, seq_fts)
  return activation(ret_fts)

In [64]:
def train_cora(fts, adj, gnn_fn, units, epochs, lr):
  lyr_1 = tf.keras.layers.Dense(units)
  lyr_2 = tf.keras.layers.Dense(7)
  
  def cora_gnn(fts, adj):
    hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
    logits = gnn_fn(hidden, adj, lyr_2, tf.nn.relu)
    return logits

  optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
  best_accuracy = 0
  for ep in range(epochs + 1):
    with tf.GradientTape() as t:
      logits = cora_gnn(fts, adj)
      loss = masked_softmax_cross_entropy(logits, labels, train_mask)

    variables = t.watched_variables()
    grads = t.gradient(loss, variables)
    optimizer.apply_gradients(zip(grads,variables))

    logis = cora_gnn(fts, adj)
    val_accuracy = masked_accuracy(logits, labels, val_mask)
    tast_accuracy = masked_accuracy(logits, labels, test_mask)

    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      print("epoch", ep, "loss = ", loss.numpy())

In [76]:
train_cora(features, adj.todense(), gnn, 32, 200, 0.01)

(2708, 1433)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 0 loss =  3.4261959
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 3 loss =  3.6815166
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 4 loss =  3.1300712
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 5 loss =  2.525069
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 6 loss =  1.6909039
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 14 loss =  1.3609345
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 15 loss =  1.3534455
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 16 loss =  1.3523811
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 17 loss =  1.3474896
(1433, 32)
(32, 7)
(1433, 32)
(32, 7)
epoch 18 loss =  1.3386761
(

KeyboardInterrupt: ignored